In [ ]:
import torch.nn as nn
import torchvision
import torch
import time
import torchvision.models as models
import torch.optim as optim

from torchvision.models import ResNet101_Weights,  MobileNet_V3_Large_Weights, EfficientNet_V2_S_Weights
from sklearn.metrics import precision_recall_fscore_support

print(f'Torch-Version {torch.__version__}')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'DEVICE: {DEVICE}')

#### **Base model preparation**

Prepapre the base models with ImageNet weight.

Exclude the original fully connected layers and add a new projection head to the base model.

In [ ]:
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNetFeatureExtractor, self).__init__()
        
        # Load the ResNet-101 model with pretrained weights
        resnet101 = models.resnet101(weights=ResNet101_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(resnet101.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveMaxPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class MobileNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(MobileNetFeatureExtractor, self).__init__()
        
        # Load the MobileNet model with pretrained weights
        Mobile_l = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(Mobile_l.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveMaxPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class EfficientNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetFeatureExtractor, self).__init__()
        
        # Load the EfficientNet model with pretrained weights
        Efficient = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(Efficient.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveMaxPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

# Define the base model structure
class SimCLR(nn.Module):
    def __init__(self, linear_eval=False):
        super().__init__()
        self.linear_eval = linear_eval
        feature_extractor = ResNetFeatureExtractor() # choose corresponding base model
        self.encoder = feature_extractor
        self.projection = nn.Sequential( 
            nn.Linear(2048, 128), # modify the input feature dimension based on models: resnet-2048, mobilenet-960, efficientnet-1024, add more models and modify them correctly
            nn.ReLU(),
            nn.Linear(128,128)
        )
    def forward(self, x):
        if not self.linear_eval:
            x = torch.cat(x, dim=0)
    
        encoding = self.encoder(x)
        projection = self.projection(encoding) 
        return projection

#### **Base model pre-training**

Define the loss function for contrastive self-supervised pre-training.

In [ ]:
# Define the batch size
BATCH_SIZE = 64

LABELS = torch.cat([torch.arange(BATCH_SIZE) for i in range(2)], dim=0)
LABELS = (LABELS.unsqueeze(0) == LABELS.unsqueeze(1)).float() # Creates a one-hot with broadcasting
LABELS = LABELS.to(DEVICE) #128,128

def cont_loss(features, temp):
    """
    The NTxent Loss.
    
    Args:
        z1: The projection of the first branch
        z2: The projeciton of the second branch
    
    Returns:
        the NTxent loss
    """
    similarity_matrix = torch.matmul(features, features.T) # 128, 128
    # discard the main diagonal from both: labels and similarities matrix
    mask = torch.eye(LABELS.shape[0], dtype=torch.bool).to(DEVICE)
    # ~mask is the negative of the mask
    # the view is required to bring the matrix back to shape
    labels = LABELS[~mask].view(LABELS.shape[0], -1) # 128, 127
    similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1) # 128, 127

    # select and combine multiple positives
    positives = similarity_matrix[labels.bool()].view(labels.shape[0], -1) # 128, 1

    # select only the negatives
    negatives = similarity_matrix[~labels.bool()].view(similarity_matrix.shape[0], -1) # 128, 126

    logits = torch.cat([positives, negatives], dim=1) # 128, 127
    labels = torch.zeros(logits.shape[0], dtype=torch.long).to(DEVICE)

    logits = logits / temp
    return logits, labels

In [ ]:
# Initialise the base model
EPOCHS = 50
simclr_model = SimCLR().to(DEVICE)
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.Adam(simclr_model.parameters())

In [ ]:
# Start pre-training
for epoch in range(EPOCHS):
    t0 = time.time()
    running_loss = 0.0
    running_accuracy = 0.0
    for i, views in enumerate(train_dl):  # Acquire the source train dataloader from data preprocessing file
        projections = simclr_model([view.to(DEVICE) for view in views])
        logits, labels = cont_loss(projections, temp=2)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        running_loss += loss / len(train_dl) # Acquire the source train dataloader from data preprocessing file

    print(f'EPOCH: {epoch+1} BATCH: {i+1} LOSS: {(running_loss):.4f}')
    print(f'Time taken: {((time.time()-t0)/60):.3f} mins\n')
    torch.save(simclr_model, "saved_model/pre-trained_model.pth") # Save a model file after pre-training

#### **Base model fine-tuning**

Reload the pre-trained base model for fine-tuning steps.

|Note. The class featurextractor and class SimCLR is necessary to reload the save model file, please rerun these sections before running the following linear evaluation section.|

In [ ]:
# Define a new base model with the same structure but replaced with a new classification head.
class LinearEvaluation(nn.Module):
    def __init__(self, model, nu_classes):
        super().__init__()
        simclr = model
        simclr.linear_eval = True
        simclr.projection = Identity()
        self.simclr = simclr
        for param in self.simclr.parameters():
            param.requires_grad = False
        self.linear = nn.Sequential(nn.Linear(2048, 128), nn.ReLU()) # modify the input dimension to match the base model structure
        self.linear2 = nn.Sequential(nn.Linear(128, nu_classes), nn.Softmax(dim=-1))
    def forward(self, x):
        encoding = self.simclr(x)
        encoding = self.linear(encoding) 
        pred = self.linear2(encoding)
        return pred

In [ ]:
# Reload the pre-trained base model and initilise it with the new classification head
new_simclrmodel = torch.load('saved_model/pre-trained_model.pth')
new_eval_model = LinearEvaluation(new_simclrmodel, nu_train_classes).to(DEVICE) # the number of train classes is defined in data preprocessing file
# note that all the layers from base models will be freezed, choose to unfreeze them for the fine-tuning step

# Provide a summary of the new model
for name, param in new_eval_model.named_parameters():
    print(name,param.requires_grad)

In [ ]:
# A sample for unfreeze selected layers for the fine-tuning
for param in new_eval_model.simclr.encoder.conv_layers[21].parameters():
        param.requires_grad = True
for param in new_eval_model.simclr.encoder.conv_layers[22].parameters():
        param.requires_grad = True

In [ ]:
# Define a function to calculate accuracy performance
def calculate_accuracy(predictions, labels):
    _, predicted = torch.max(predictions, 1)
    correct = (predicted == labels).sum().item()
    return correct

In [ ]:
# Initilise the new_eval model
EPOCHS = 50
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.Adam(new_eval_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.2) # A step learning rate decrease function, start from 40 epochs

In [ ]:
# Use an array to store all the predictions and ground truth labels for evaluation
train_predicted_results = []
train_ground_truth_labels = []

# Start model training
for epoch in range(EPOCHS):
    t0 = time.time()
    running_loss = 0.0
    running_accuracy = 0.0
    for i, element in enumerate(train_dl): # Acquire the target train dataloader from data preprocessing file
        image, label = element
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        pred = new_eval_model(image)
        loss = criterion(pred, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        acc = (pred.argmax(dim=1) == label).float().mean()
        running_accuracy += acc / len(train_dl)  # Acquire the target train dataloader from data preprocessing file
        running_loss += loss / len(train_dl)  # Acquire the target train dataloader from data preprocessing file

        # Append predicted results and ground truth labels to the arrays
        train_predicted_results.extend(torch.argmax(pred, dim=1).cpu().numpy())
        train_ground_truth_labels.extend(label.cpu().numpy())
        
        #If with validation dataset, use this following code
        #with torch.no_grad():
            #epoch_val_accuracy = 0
            #epoch_val_loss = 0
        
        #for data, label in val_dl:

            #data = data.to(DEVICE)
            #label = label.to(DEVICE)

            #val_output = new_eval_model(data)
            #val_loss = criterion(val_output, label)
            
            #acc = (val_output.argmax(dim=1) == label).float().mean()
            #epoch_val_accuracy += acc / len(val_dl)
            #epoch_val_loss += val_loss / len(val_dl)
    scheduler.step()
    current_lr = optimizer.param_groups[0]["lr"]    
    train_precision, train_recall, train_f1_score, _ = precision_recall_fscore_support(train_ground_truth_labels, train_predicted_results, average=None)
    print(f'EPOCH: {epoch+1} BATCH: {i+1} LOSS: {(running_loss):.4f} ACCURACY: {(running_accuracy*100):.4f} LEARNING_RATE: {(current_lr):.8f}')
    print(f'Precision: {train_precision} Recall:{train_recall} F1_score:{train_f1_score}')
    #print(f'Val_loss : {epoch_val_loss:.4f} Val_acc: {epoch_val_accuracy:.4f}')
    print(f'Time taken: {((time.time()-t0)/60):.3f} mins \n')


In [ ]:
# Use an array to store all the predictions and ground truth labels for evaluation
predicted_results = []
ground_truth_labels = []
# Evaluate the model and calculate accuracy
test_model = new_eval_model
test_model.eval()  # Set the model to evaluation mode
test_correct = 0
test_samples = 0
test_loss = 0.0
test_recall = 0.0
test_precision = 0.0
test_num = 0

with torch.no_grad():
    for i, test_elements in enumerate(test_dl):  # Acquire the target test dataloader from data preprocessing file
        test_image, test_label = test_elements
        test_image = test_image.to(DEVICE)
        test_label = test_label.to(DEVICE)
        test_output = test_model(test_image)

        test_step_correct = calculate_accuracy(test_output, test_label)
        test_step_loss = criterion(test_output, test_label)
        _, predicted = torch.max(test_output, 1)
        test_loss += test_step_loss.item()
        test_correct += test_step_correct
        test_samples += test_label.size(0)
        test_num += 1
    
        # Append predicted results and ground truth labels to the arrays
        predicted_results.extend(torch.argmax(test_output, dim=1).cpu().numpy())
        ground_truth_labels.extend(test_label.cpu().numpy())
        
# Output model performance with selected evaluation metrics
test_accuracy = test_correct / test_samples
test_loss = test_loss / test_num
precision, recall, f1_score, _ = precision_recall_fscore_support(ground_truth_labels, predicted_results, average=None)
print("Model loss:", test_loss)
print("Model accuracy:", test_accuracy)
print("Model recall:", recall)
print("Model precision:", precision)
print("Model f1:", f1_score)